In [1]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from   torchtext.vocab import GloVe
from   torchtext.legacy.data import Field
from   torch.optim import lr_scheduler

##### 创建数据集

In [2]:
TEXT                 = torchtext.legacy.data.Field(lower=True,fix_length=200,batch_first=False)
LABEL                = torchtext.legacy.data.Field(sequential=False)
train,test           = torchtext.legacy.datasets.IMDB.splits(TEXT,LABEL)
TEXT.build_vocab(train,max_size=10000,min_freq=10,vectors=None)
LABEL.build_vocab(train)
train_iter,test_iter = torchtext.legacy.data.BucketIterator.splits((train,test),batch_size=256)

##### 创建模型

In [3]:
embedding_dim = 100
hidden_size   = 300

In [4]:
class RNN_Encoder(nn.Module):
    '''模型将对评论依次进行读取并输出最后状态'''
    def __init__(self,input_dim,hidden_size):
        '''
            input_dim   表示输入序列的长度
            hidden_size 输出的中间隐藏层的状态数
        '''
        super(RNN_Encoder,self).__init__()
        ''' 初始化RnnCell '''
        self.rnn = nn.GRUCell(input_dim,hidden_size)
        
    def forward(self,inputs):
        ''' inputs  输入(文本/数值 etc)的序列                     '''
        '''         shape:seq(序列本身),batch(所属batch的大小)    '''
        '''              :embedding(每一个单词所映射到的张量的长度) '''
        bz = inputs.shape[1]  # bz==batch_size
        ''' 初始状态值 每一个单词到达一个层的时候都会产生一个状态       '''
        ht = torch.zeros((bz, hidden_size)).cuda()
        for word in inputs:
            '''RNN的输入是Embedding层的输出'''
            ht = self.rnn(word,ht)
        '''ht是RNN对句子整体理解数值化表示'''
        return ht

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        '''对单词进行Embedding操作参数为词表中单词数目，目标向量空间维度'''
        self.em  = nn.Embedding(len(TEXT.vocab.stoi),embedding_dim)
        '''hidden_size是一个超参数可以自定义'''
        self.rnn = RNN_Encoder(embedding_dim,hidden_size)
        self.fc1 = nn.Linear(hidden_size,256)
        self.fc2 = nn.Linear(256,3)#三分类模型
    
    def forward(self,x):
        x   = self.em(x)  # 词嵌入表示
        x   = self.rnn(x) # 使用ct作为最终RNN处理序列数据的结果
        x   = self.fc1(x)
        x   = F.relu(x)
        x   = self.fc2(x)
        return x

In [6]:
model      = Net() 
if torch.cuda.is_available():
    model.to('cuda')
loss_fn    = nn.CrossEntropyLoss()
optimizer  = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs     = 50
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

In [7]:
def fit(epoch, model, trainloader, testloader):
    correct      = 0
    total        = 0
    running_loss = 0
    
    model.train()
    for b in trainloader:
        x, y     = b.text, b.label
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred   = model(x)
        loss     = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred        = torch.argmax(y_pred, dim=1)
            correct      += (y_pred == y).sum().item()
            total        += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc  = correct / total
        
        
    test_correct      = 0
    test_total        = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for b in testloader:
            x, y   = b.text, b.label
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss   = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct      += (y_pred == y).sum().item()
            test_total        += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc  = test_correct / test_total
    
        
    print('epoch: ', epoch,'loss： ', round(epoch_loss, 3),'accuracy:', round(epoch_acc, 3),'test_loss： ', round(epoch_test_loss, 3),'test_accuracy:', round(epoch_test_acc, 3))
        
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

In [8]:
for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,model,train_iter,test_iter)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`